<a href="https://colab.research.google.com/github/TimDemetriades/PythonSudokuSolver/blob/master/PythonSudokuSolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Python Solver**

In [29]:
import time

board = [               # Sudoku Board
 [9,0,0,0,0,0,5,1,0],   # 1
 [0,7,0,0,8,0,0,0,9],   # 2
 [5,0,0,1,0,9,2,7,8],   # 3
 [2,5,0,0,0,7,8,0,1],   # 4
 [1,0,0,0,5,0,0,0,0],   # 5
 [3,0,0,0,9,0,0,0,0],   # 6
 [0,0,0,3,1,5,0,4,0],   # 7
 [0,0,0,0,0,0,0,0,0],   # 8
 [7,1,4,0,2,0,0,8,0]    # 9
]

def displayBoard(board):
  """
  prints the board
  :parameter board: 2d list of integers
  :return: None
  """

  for i in range(len(board)):   # rows
    if i % 3 == 0 and i != 0:
      print("-------------------------")  # horiontal lines

    for j in range(len(board[0])):    # columns
      if j % 3 == 0 and j != 0:
        print(" | "+ ' ', end = '')    # vertical lines (end = '' is used to not end with newline (\n))

      if j == 8:    # last value in row
        print(board[i][j])
      else:
        print(str(board[i][j]) + ' ', end = '')   # str is so we can add the space

def findZero(board):
  """
  finds a zero in the board
  :parameter board: partially complete board
  :return: (integer, integer) row column
  """

  for row in range(len(board)):
    for column in range(len(board[0])):
      if board[row][column] == 0: 
        return(row, column)    # row, column

  return None   # needed since otherwise wil return position of zero

def validNumber(board, number, position):
  """
  checks if number is valid in board
  :parameter board: 2d list of integers
  :parameter number: integer
  :parameter position: position of number (row, column)
  :return: bool
  """

  # First check row
  for i in range(9):
    if board[position[0]][i] == number and i != position[1]:    # check if equal in row and not position we just entered number
      return False

  # Then check column
  for i in range(9):
    if board[i][position[1]] == number and i != position[1]:   # check if equal in column and not position we just entered number
      return False

  # Finally, check square
  square_x = position[1] // 3   # floor division (will give 0, 1, or 2)
  square_y = position[0] // 3   # floor division (will give 0, 1, or 2)

  for i in range(square_y * 3, square_y * 3 + 3):   # from beginning to end of square row
    for j in range(square_x * 3, square_x * 3 + 3):   # from beginning to end of square column
      if board[i][j] == number and (i, j) != position:
        return False

  return True   # if all 3 tests do not return false, return true since number is valid

def solver(board):
  """
  solves sudoku board using backtracking
  :parameter board: 2d list of integers
  :return: solution
  """

  # Base case of recusion, checks if any zeroes are left
  find = findZero(board)   # checks for zeroes
  if find != None:    # found zero
    row, column = find    # find is a tuple e.g. (x, y)
  else:   # no zeroes left; find is none
    return True    # board is done

  for number in range(1,10):   # tries numbers 1 to 9
    if validNumber(board, number, (row, column)) is True:   # if number valid
      board[row][column] = number   # inserts number

      if solver(board) is True:   # if no zeroes left
        return True  # board is solved

      else:   # zeroes found
        board[row][column] = 0    # board not solved, reset number to zero 
        continue
    
    else:   # if number not valid
      continue


if __name__ == "__main__":
  print("Un-solved Board")
  displayBoard(board)

  startTime = time.time()

  solver(board)

  elapsedTime = time.time() - startTime
  elapsedTime = str(round(elapsedTime, 4))

  print("")
  print("Solved Board")
  displayBoard(board)

  print("")
  print("Time elapsed: ", elapsedTime, " seconds")



Un-solved Board
9 0 0  |  0 0 0  |  5 1 0
0 7 0  |  0 8 0  |  0 0 9
5 0 0  |  1 0 9  |  2 7 8
-------------------------
2 5 0  |  0 0 7  |  8 0 1
1 0 0  |  0 5 0  |  0 0 0
3 0 0  |  0 9 0  |  0 0 0
-------------------------
0 0 0  |  3 1 5  |  0 4 0
0 0 0  |  0 0 0  |  0 0 0
7 1 4  |  0 2 0  |  0 8 0

Solved Board
9 2 8  |  7 6 3  |  5 1 4
4 7 1  |  5 8 2  |  6 3 9
5 6 3  |  1 4 9  |  2 7 8
-------------------------
2 5 9  |  4 3 7  |  8 6 1
1 4 7  |  6 5 8  |  9 2 3
3 8 6  |  2 9 1  |  4 5 7
-------------------------
8 9 2  |  3 1 5  |  7 4 6
6 3 5  |  8 7 4  |  1 9 2
7 1 4  |  9 2 6  |  3 8 5

Time elapsed:  0.0768  seconds


**Python Solver for NY Times Daily Sudoku**

In [31]:
import requests
import re
import ast
import time
from bs4 import BeautifulSoup

# Get page
page = requests.get('https://www.nytimes.com/puzzles/sudoku')
if page.status_code != 200:   # 200 = good
  print("Page request failed :(")
else:
  print("Grabbing Today's NYTimes Sudoku Puzzle\n")

# Get content
content = page.content 

# Parse HTML
soup = BeautifulSoup(content, 'html.parser')

# Grab part that has puzzles
results = soup.find_all('script', attrs={'type':'text/javascript'})[0].text  

# Get easy puzzle
split = results.split('"puzzle":',2)[1]   # get part of puzzle after first occurance of 'puzzle'
puzzleEasy = split.split(',"solution"',1)[0]    # get part of puzzle before first occurance of 'solution'

# Get medium puzzle
split = results.split('"medium":',1)[1]
split2 = split.split('"puzzle":',2)[1]
puzzleMedium = split2.split(',"solution"',1)[0]

# Get hard puzzle
split = results.split('"hard":',1)[1]
split2 = split.split('"puzzle":',2)[1]
puzzleHard = split2.split(',"solution"',1)[0]

# Convert strings into lists
puzzleEasyList = ast.literal_eval(puzzleEasy) 
puzzleMediumList = ast.literal_eval(puzzleMedium) 
puzzleHardList = ast.literal_eval(puzzleHard) 

def makeListofLists(quantity, listGiven):
  """
  takes a list and makes it into a list of lists
  :param quantity: how many items you want in each list
  :param listGiven: list you want to make into list of lists
  """
  i = 0
  newList = []
  while i < len(listGiven):
    newList.append(listGiven[i:i+quantity])
    i += quantity
  return(newList)

def displayBoard(board):
  """
  prints the board
  :parameter board: 2d list of integers
  :return: None
  """

  for i in range(len(board)):   # rows
    if i % 3 == 0 and i != 0:
      print("-------------------------")  # horiontal lines

    for j in range(len(board[0])):    # columns
      if j % 3 == 0 and j != 0:
        print(" | "+ ' ', end = '')    # vertical lines (end = '' is used to not end with newline (\n))

      if j == 8:    # last value in row
        print(board[i][j])
      else:
        print(str(board[i][j]) + ' ', end = '')   # str is so we can add the space

def findZero(board):
  """
  finds a zero in the board
  :parameter board: partially complete board
  :return: (integer, integer) row column
  """

  for row in range(len(board)):
    for column in range(len(board[0])):
      if board[row][column] == 0: 
        return(row, column)    # row, column

  return None   # needed since otherwise wil return position of zero

def validNumber(board, number, position):
  """
  checks if number is valid in board
  :parameter board: 2d list of integers
  :parameter number: integer
  :parameter position: position of number (row, column)
  :return: bool
  """

  # First check row
  for i in range(9):
    if board[position[0]][i] == number and i != position[1]:    # check if equal in row and not position we just entered number
      return False

  # Then check column
  for i in range(9):
    if board[i][position[1]] == number and i != position[1]:   # check if equal in column and not position we just entered number
      return False

  # Finally, check square
  square_x = position[1] // 3   # floor division (will give 0, 1, or 2)
  square_y = position[0] // 3   # floor division (will give 0, 1, or 2)

  for i in range(square_y * 3, square_y * 3 + 3):   # from beginning to end of square row
    for j in range(square_x * 3, square_x * 3 + 3):   # from beginning to end of square column
      if board[i][j] == number and (i, j) != position:
        return False

  return True   # if all 3 tests do not return false, return true since number is valid

def solver(board):
  """
  solves sudoku board using backtracking
  :parameter board: 2d list of integers
  :return: solution
  """

  # Base case of recusion, checks if any zeroes are left
  find = findZero(board)   # checks for zeroes
  if find != None:    # found zero
    row, column = find    # find is a tuple e.g. (x, y)
  else:   # no zeroes left; find is none
    return True    # board is done

  for number in range(1,10):   # tries numbers 1 to 9
    if validNumber(board, number, (row, column)) is True:   # if number valid
      board[row][column] = number   # inserts number

      if solver(board) is True:   # if no zeroes left
        return True  # board is solved

      else:   # zeroes found
        board[row][column] = 0    # board not solved, reset number to zero 
        continue
    
    else:   # if number not valid
      continue

def getDifficulty(prompt):
  while True:
    try:
      value = input(prompt)
    except ValueError:
      print("Invalid Selection")
      continue

    if value not in ('easy', 'medium', 'hard'):
      print("Invalid Selection")
      continue
    else:
      break
  return value

################################################################################

if __name__ == "__main__":

  print("NYTimes Sudoku Solver\n")
  print("Please choose a diffulty")
  difficulty = getDifficulty("Type either easy, medium, or hard: ")

  if difficulty == 'easy':
    board = makeListofLists(9, puzzleEasyList)
  elif difficulty == 'medium':
    board = makeListofLists(9, puzzleMediumList)
  elif difficulty == 'hard':
    board = makeListofLists(9, puzzleHardList)

  print("")
  print("Un-solved Board")
  displayBoard(board)

  startTime = time.time()

  solver(board)

  elapsedTime = time.time() - startTime
  elapsedTime = str(round(elapsedTime, 4))

  print("")
  print("Solved Board")
  displayBoard(board)

  print("")
  print("Time elapsed: ", elapsedTime, " seconds")


Grabbing Today's NYTimes Sudoku Puzzle

NYTimes Sudoku Solver

Please choose a diffulty
Type either easy, medium, or hard: easy

Un-solved Board
6 0 8  |  0 0 0  |  2 0 7
0 0 0  |  0 8 2  |  0 4 3
7 0 0  |  0 3 0  |  0 0 5
-------------------------
0 7 2  |  9 1 0  |  3 8 0
8 4 3  |  0 0 0  |  9 2 0
9 0 0  |  3 2 8  |  7 0 0
-------------------------
0 0 9  |  5 0 0  |  0 3 2
0 1 0  |  8 0 3  |  0 6 0
0 5 0  |  0 0 1  |  4 0 0

Solved Board
6 3 8  |  4 5 9  |  2 1 7
1 9 5  |  7 8 2  |  6 4 3
7 2 4  |  1 3 6  |  8 9 5
-------------------------
5 7 2  |  9 1 4  |  3 8 6
8 4 3  |  6 7 5  |  9 2 1
9 6 1  |  3 2 8  |  7 5 4
-------------------------
4 8 9  |  5 6 7  |  1 3 2
2 1 7  |  8 4 3  |  5 6 9
3 5 6  |  2 9 1  |  4 7 8

Time elapsed:  0.0038  seconds
